In [1]:
import pandas as pd
import numpy as np
import pickle
from collections import defaultdict

In [2]:
#collaborative imports
from surprise import Dataset
from surprise import Reader
from surprise import dump
from surprise import accuracy
from surprise import SVD
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV
from surprise.model_selection import KFold
from surprise.model_selection.split import train_test_split

In [3]:
df = pd.read_csv('../data/top_ratings.csv') #dataframe containing reviews for top beers
df = df.dropna()
df = df[~(df.score>5)]
df['uid'] = df.groupby('username').ngroup()
# user_df = df.pivot(index='uid',columns='beer_id',values='score')
# user_df

In [4]:

class CollabRecommender():
    '''
    SVD Collaborative-Based Beer Recommender
    '''
    def __init__(self, df):
        reader = Reader(rating_scale=(1,5))
        self.data = Dataset.load_from_df(df[['uid','beer_id','score']],reader)
        self.trainset, self.testset = train_test_split(self.data,test_size=.20)
        self.model = self.load_obj('model')
        try:
            self.predictions = self.load_obj('predictions')
        except:
            print('Unable to load predictions, please use .predict() method.')
        try:
            self.model = self.load_obj('model')
        except:
            print('Unable to load model, please use the .fit() method.')

    def fit(self, model=SVD, n_factors=100, n_epochs=20, lr_all=0.005, reg_all=0.02):
        self.model = model(n_factors=n_factors, n_epochs=n_epochs, lr_all=lr_all, reg_all=reg_all, verbose=True)
        self.model.fit(self.trainset)
        self.save_obj(self.model, 'model')
    
    def predict(self):
        
        print('Generating predictions...')
        self.predictions = self.model.test(self.testset)
    
    def get_top_n(self, n=10):
        '''
        Return the top-N recommendation for each user from a set of predictions.
        Input:
            predictions(list of Prediction objects): The list of predictions, as
                returned by the test method of an algorithm.
            n(int): The number of recommendation to output for each user. Default
                is 10.

        Returns:
        A dict where keys are user (raw) ids and values are lists of tuples:
            [(raw item id, rating estimation), ...] of size n.
        '''

        # First map the predictions to each user.
        top_n = defaultdict(list)
        for uid, iid, true_r, est, _ in self.predictions:
            top_n[uid].append((iid, est))

        # Then sort the predictions for each user and retrieve the k highest ones.
        for uid, user_ratings in top_n.items():
            user_ratings.sort(key=lambda x: x[1], reverse=True)
            top_n[uid] = user_ratings[:n]

        return top_n
    

    #pickle helper functions
    def save_obj(self,obj, name):
        with open('../obj/'+ name + '.pkl', 'wb') as f:
            pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

    def load_obj(self,name):
        with open('../obj/' + name + '.pkl', 'rb') as f:
            return pickle.load(f)     


In [5]:
colab_r = CollabRecommender(df)

In [20]:
predictions = colab_r.predict()

Generating predictions...


In [8]:
def precision_recall_at_k(predictions, k=10, threshold=4):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

In [15]:
precisions, recalls = precision_recall_at_k(colab_r.predictions,k=5,threshold=4)

In [16]:
print(sum(prec for prec in precisions.values()) / len(precisions))
print(sum(rec for rec in recalls.values()) / len(recalls))

0.7297250536157246
0.6057569603504699


In [6]:
top_ratings = colab_r.get_top_n()

In [7]:
top_ratings[39309]

[(16909, 4.329816504640543),
 (10325, 4.291671178021917),
 (1577, 4.263637058932049),
 (18862, 4.195485977571795),
 (49472, 4.182998319583871),
 (705, 4.175620853459322),
 (3916, 4.137831205635079),
 (6646, 4.119690070148591),
 (30, 4.1029104887799095),
 (25852, 4.081892898481363)]

In [37]:
sim_users = [170, 203,  87, 169, 190]
beers={}
for user in sim_users:
    for item in top_ratings[user]:
        if len(item)>0:
            beers[item[0]]=item[1]

beers


{1372: 4.062112052778878, 7463: 4.1464328095279495, 22343: 3.8146801284069456}

In [41]:
beers = dict(sorted(beers.items(),key=lambda item: item[1], reverse=True))
beers

{7463: 4.1464328095279495, 1372: 4.062112052778878, 22343: 3.8146801284069456}

In [38]:
max(beers,key=beers.get)

7463

In [11]:
def get_Iu(uid):
    """ return the number of items rated by given user
    args: 
      uid: the id of the user
    returns: 
      the number of items rated by the user
    """
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # user was not part of the trainset
        return 0
    
def get_Ui(iid):
    """ return number of users that have rated given item
    args:
      iid: the raw id of the item
    returns:
      the number of users that have rated the item.
    """
    try: 
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0
    
pred_df = pd.DataFrame(predictions, columns=['username', 'beer_id', 'rui', 'est', 'details'])
pred_df['Iu'] = pred_df.username.apply(get_Iu)
pred_df['Ui'] = pred_df.beer_id.apply(get_Ui)
pred_df['err'] = abs(pred_df.est - df.rui)
best_predictions = pred_df.sort_values(by='err')[:10]
worst_predictions = pred_df.sort_values(by='err')[-10:]

NameError: name 'predictions' is not defined

In [6]:
pu = colab_r.model.pu
qi = colab_r.model.qi
bu = colab_r.model.bu
bi = colab_r.model.bi

In [26]:
bi.shape

(1000,)

In [37]:
avg=df.score.mean()
pq = np.dot((bi[:,None]+qi),(bu[:,None]+pu).T)
prediction = (avg+pq).T
prediction.shape

(122278, 1000)

In [41]:
prediction[prediction>5]=5
prediction[prediction<0]=0

In [47]:
top_recs = prediction.argsort()
top_recs[0][:10]

array([  0, 629, 628, 626, 625, 624, 622, 616, 610, 605])

In [50]:
df.iloc[0]

beer_id      125646
username    _dirty_
text               
score           4.5
uid           68954
Name: 0, dtype: object